In [11]:
import string
import json
import pandas as pd
import re

raw_df = pd.read_excel("Chat Conv Part B.xlsx")
df_grouped = raw_df.groupby('chat_id')


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data



def save_json_file(output_path, data):
   with open(output_path, 'w') as file:
        json.dump(data, file, indent=4)

def split_json_file(json_data):
    
    json_data1, json_data2 = {}, {}
    mid_index = len(json_data) // 2
    
    for key, value in json_data.items():
        if len(json_data1) < mid_index:
            json_data1[key] = value
        else:
            json_data2[key] = value

    return json_data1, json_data2



In [12]:
json_data = {}

not_enough = {}
counter = 0
# Loop through grouped DataFrame (replace with your grouped DataFrame name)
for group_name, group_df in df_grouped: 
    counter += 1
 
    if (counter % 500 == 0): 
        print(counter)
        
    if ((group_df.iloc[0]['is_english']) == False):
        continue
        
    
    
    
    welcome_message = 0
    
    # Reset the index to start from 0
    group_df = group_df.reset_index(drop=True)
    dialog_dict = {"number_of_turns": 0,
                            "annotator_id": None,
                            "dialog": {}}

    # Add the welcome message if it exists
    if str(group_df.iloc[0]['welcome_message']) != "nan":
        dialog_dict["number_of_turns"] += 1
        welcome_message = 1
        dialog_dict["dialog"][0] = {
            "turn_num": 0,
            "original_question": group_df.iloc[0]['welcome_message'],
            "answer": "",
        }

    turns_counter = 0

    # Loop through the DataFrame by 2 rows at a time
    for index in range(0, len(group_df), 2):
        
         # Increment the number of turns
        dialog_dict["number_of_turns"] += 1
    
        # Get the current row
        row1 = group_df.iloc[index]
       
        if index + 1 < len(group_df):
            row2 = group_df.iloc[index + 1]
        else:
            row2 = None  # If the number of rows is odd, the last pair will have None for the second row
    
        # Calc the turn number
        turn_num = int(index / 2) + welcome_message

        # Add the two rows to the JSON
        dialog_dict["dialog"][turn_num] = {
        "turn_num": int(index / 2) + welcome_message,
        "original_question": str(row1['message']),
        "answer": str(row2['message']) if row2 is not None else "None",
        }

        if index + welcome_message> 0:
            dialog_dict["dialog"][turn_num]["requires_rewrite"] = None
            dialog_dict["dialog"][turn_num]["enough_context"] = None
            turns_counter += 1
            

    if turns_counter > 0:
        json_data[group_name] = dialog_dict
    else:
        not_enough[group_name] = dialog_dict
        


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [13]:
save_json_file("agent_conv_3.json", json_data)